<a href="https://colab.research.google.com/github/josephmhernandez/SeizureDetection/blob/master/SeizureDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Data

In [0]:
#Loading openPose.


## openPose

In [0]:
#Load Open Pose
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


### Creating action landmarks

In [0]:
#Connect to Google Drive. 
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#Seizure Videos openPose output generation. 
import os
import numpy as np

data = 'gdrive/My\ Drive/Seizure\ DL/Dataset/SeizureVideos/Vids/'
typeVid = 'Seizure'
jdest = 'gdrive/My\ Drive/Seizure\ DL/Dataset/SeizureVideos/Data/'

pythonPushupsPath = 'gdrive/My Drive/Seizure DL/Dataset/SeizureVideos/'

list_vids = np.array(os.listdir(pythonPushupsPath + 'Vids/'))
i = 1
err_list = []
for filename in list_vids:
  try:
    typeVid = filename[:len(filename) - 4]
    
    newFolder = pythonPushupsPath + typeVid + "vidOutput" + str(i)

    newvidInput = data + filename
    newvidOutput = jdest + typeVid + "vidOutput" + str(i) + ".avi"
    newJsonOutput = jdest + typeVid + "vidOutput" + str(i) + '/'

    !./openpose/build/examples/openpose/openpose.bin --video {newvidInput} --write_video {newvidOutput} --write_json {newJsonOutput} --model_folder openpose/models --disable_blending --display 0 --number_people_max 1 --scale_number 4 --scale_gap 0.25 --keypoint_scale 3

  except: 
    err_list.append(filename)
  i += 1

print(err_list)


In [0]:
#Organizing videos and generating openPose output. 
import os
import numpy as np

actions = ['Jump', 'Laugh', 'PushUps', 'Sit', 'Situp']
# actions = ['Sit', 'Situp'] 'Brush_hair', 'Clap', 'Climbing_stairs', 'Drink', 'Eat', 
# actions = ['Clap']

for a in actions:
  typeVid = a
  data = 'gdrive/My\ Drive/Seizure\ DL/Dataset/NonSeizureVideos/' + typeVid + '/Vids/'
  jdest = 'gdrive/My\ Drive/Seizure\ DL/Dataset/NonSeizureVideos/' + typeVid + '/Data/'


  pythonPushupsPath = 'gdrive/My Drive/Seizure DL/Dataset/NonSeizureVideos/' + typeVid + '/' 

  try:
    list_vids = np.array(os.listdir(pythonPushupsPath + 'Vids/'))
  except:
    print('bad Action:' + a)
    
  i = 1
  err_list = []
  for filename in list_vids:
    try:
      src = pythonPushupsPath + 'Vids/'+ filename
      
      newFolder = pythonPushupsPath + typeVid + "vidOutput" + str(i)

      newvidInput = data + typeVid + "vid" +str(i) + ".avi"
      newvidOutput = jdest + typeVid + "vidOutput" + str(i) + ".avi"
      newJsonOutput = jdest + typeVid + "vidOutput" + str(i) + '/'

      !./openpose/build/examples/openpose/openpose.bin --video {newvidInput} --write_video {newvidOutput} --write_json {newJsonOutput} --model_folder openpose/models --disable_blending --display 0 --number_people_max 1 --scale_number 4 --scale_gap 0.25 --keypoint_scale 3

    except: 
      err_list.append(filename)
    i += 1

# Data Preparation

## Data Formatting

In [0]:
#Produce (80, 50) np arrays and store them into npy documents. One document per each action  so ~200 documents. 
import numpy as np
import os
import random
import json

#Mount google drive. 
#Connect to Google Drive. 
from google.colab import drive
drive.mount('/content/gdrive')

root_path = 'gdrive/My Drive/Seizure DL/Data_2/'

send_path = 'gdrive/My Drive/Seizure DL/Data_3/'

#Get three folders that need to be itterated through (Train, Test, Valid) 
folders = ['Validation', 'Train', 'Test']
#Get list of all actions

#Iterate through all actions (directories)
count = 0
for rootFolderName in folders: #[Train, test, validation]
    rootFolderName = 'Test'
    path = root_path + rootFolderName + '/'
    actions = np.array(os.listdir(path))
    print('start ' + rootFolderName)
    for a in actions:

        path2 = path + a + '/'
        jsonFiles = np.array(os.listdir(path2))
        rtnList = np.zeros((80,50)) 
        jsonFiles.sort()
        for i,jsonFile in enumerate(jsonFiles): 
            with open(path2 + jsonFiles[i]) as jsonOutput:
                data = json.load(jsonOutput)
                try:
                    bodyLandmarks = data['people'][0]['pose_keypoints_2d']
                    #Get rid of the confidence of points
                    bodyLandmarks = [xx for i, xx in enumerate(bodyLandmarks) if i%3 !=2]
                    rtnList[i,] = bodyLandmarks
                except:
                    continue
            #Only extracts the first 80 frames/time_steps
            if (i > 78):
                break
        
        #Convert (80, 50) np.array to a .npy file
        np.save(send_path + rootFolderName + '/' + a + '.npy', rtnList)
        count += 1 
        if count % 5 == 0:
            print(count)

    print('Finished ' + rootFolderName)


## Function Generator

In [0]:
#Allows multiprocessing during training. 
import numpy as np
import random
import json
import os
import keras

class DataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels, location="location__", batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        #'Initialization'

        #(number of channels, number of classes, batch size)
        self.dim = dim
        
        self.batch_size = batch_size

        #Dictionary whwere labels[ID] = label
        self.labels = labels
        
        #List of 
        self.list_IDs = list_IDs

        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.location_data = location


    def on_epoch_end(self):
        #Updates indexes after each epoch
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim))#, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            # data = np.load(self.location + ID + ) 
            X[i,] = np.load(self.location_data + ID) 
           
            # strInput = self.location_data + '/' + ID + '/'
            # # print('strInput: ', strInput)
            # X[i,] = self.getLandmarks(strInput).reshape(self.dim[0], self.dim[1])#, self.n_channels)

            # Store class
            y[i] = self.labels[ID]
            # print(y.shape)
        return X, y

        # return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

    def __len__(self):
        #'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        #'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def getLandmarks(self, pyPath):
        #Input -> Location of json files (openPose output).
        #Return -> list of the first 80 outputs in the JSON folders with 75 body landmarks. 
        rtnList = np.zeros((80,50))
        jsonFiles = np.array(os.listdir(pyPath))
        # print('jsonFiles List:', jsonFiles)
        jsonFiles.sort()

        for i,fileName in enumerate(jsonFiles): 
            with open(pyPath + jsonFiles[i]) as jsonOutput:
                data = json.load(jsonOutput)
                try:
                    bodyLandmarks = data['people'][0]['pose_keypoints_2d']
                    #Get rid of the confidence of points
                    bodyLandmarks = [xx for i, xx in enumerate(bodyLandmarks) if i%3 !=2]
                    rtnList[i] = bodyLandmarks
                except:
                    continue
            #Only extracts the first 80 frames/time_steps
            if (i > 78):
                break

        #Should return (frames = 80, landmarks/features = 75)
        return rtnList

## Editing Sparse Matricies

In [0]:
import numpy as np
import random 
import os 

#For no more tensor flow warnings. 
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

def fill_sparse_data(read_path, write_path):
    #Some of the openPose outputs don't capture 'continuous' data for each landmark.
    #The idea for solving this is to replace empty and sparse frames with more 
    #   continuous frame data. For example let X be an array of size 80 (80 timestamps)
    #   X = [0, 0, 0, 1, 2, 3, 0, 0, 0, 3, 3, 4, 0, 5, 0, 0, 0, ....] 
    #   I will assume that these landmarks haven't gone off screen because there
    #   coordinates for thes landmarks as the index increases. 
    #   Outputted feature vector:
    #   new_X = 
    #       [0, 0, 0, 1, 2, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, ....] 
    # Note: Maybe would rather format end of the arrays to that there won't be 
    #       incorrect values if the landmark is off the screen and doesn't come
    #       back. 
    #       Example:
    #   new_new_X:
    #       [0, 0, 0, 1, 2, 3, 3, 3, 3, 3, 3, 4, 4, 5, 0, 0, 0, .... , 0] 
    #   ... To keep the integrity of the features if the landmark leaves the screen at the end. 

    #Foramt read/write paths.
    if(read_path[len(read_path) - 1] != '/'):
        read_path += '/'
    if(write_path[len(write_path) - 1] != '/'):
        write_path += '/'
    
    files = os.listdir(path)
    
    for file_name in files: 
        data = np.load(path + file_name)
        rtnData = np.zeros((len(data), 50))

        _prevValues = np.zeros(50)       
        for j,features in enumerate(data):
            _values = np.zeros(50)
            for i,f in enumerate(features): 
                #Iterate over 50 values. 
                if f == 0:
                    _values[i] = _prevValues[i]
                else:
                    #f != 0:
                    _values[i] = f
                    _prevValues[i] = f

            rtnData[j,] = _values


        np.save(write_path + file_name, rtnData)
        
    print('Finished writing.')
        




In [0]:
#Show the sparsity in openPose landmark featuers using Data_3 files. 
import numpy as np
import os 

py_train_path = 'gdrive/My Drive/Seizure DL/All_Data_Non_Sparse/Train/'
# py_test_path = 'gdrive/My Drive/Seizure DL/Data_3/Test/'
# py_validation_path = 'gdrive/My Drive/Seizure DL/Data_3/Validation/'

partition = {}
partition['train'] = np.array(os.listdir(py_train_path))
# partition['test'] = np.array(os.listdir(py_test_path))
# partition['validation'] = np.array(os.listdir(py_validation_path))

print('Percent of all zero features (each of the 50 features are 0) out of 80 timestamps:')
for p in partition:
    print(p + ' data:')

    #Just to get the root path correct for each partition.
    _temp_path = None
    if (p is 'train'):
        _temp_path = py_train_path
    elif(p is 'test'):
        _temp_path = py_test_path
    else:
        _temp_path = py_validation_path

    for doc in partition[p]:
        count_time_stamps = 0
        zero_sum = 0 

        _data = np.load(_temp_path + doc)

        for time_stamp in _data:
            count_time_stamps += 1
            if(sum(time_stamp) == 0):
                zero_sum += 1

        print(doc + ' -> ' + str(zero_sum / count_time_stamps * 100))
    print('\n\n') 

# Training

## Metrics


In [0]:
#Define metrics to analyze training/testing/validataion
#Build the model function. 
from keras import models
from keras import layers
from keras import optimizers

#Build the model function. 
from keras import models
from keras import layers
import keras_metrics as km
import keras.backend as K

def build_model_2():
    seq_model = models.Sequential()
    seq_model.add(layers.LSTM(200, activation='relu', input_shape=(80, 50), return_sequences = False))
    # seq_model.add(layers.LSTM(40, activation='relu', return_sequences = False))
    seq_model.add(layers.Dropout(.2))
    seq_model.add(layers.Dense(1, activation='sigmoid'))
    
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)

    seq_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy', f1_score])

    return seq_model

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def build_model_3():
    seq_model = models.Sequential()
    seq_model.add(layers.LSTM(1000, activation='relu', input_shape=(80, 50), return_sequences = False))
    # seq_model.add(layers.LSTM(40, activation='relu', return_sequences = False))
    seq_model.add(layers.Dropout(.35))
    seq_model.add(layers.Dense(1, activation='sigmoid'))
    
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.3, nesterov=True)

    seq_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy', f1_score, precision_m, recall_m])

    return seq_model

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


#Define F1 score. 
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

## K-Fold Cross Validation

In [0]:
import numpy as np
import random 
import os 

#For no more tensor flow warnings. 
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')



In [0]:
#Build the model function. 
from keras import models
from keras import layers
from keras import optimizers

import random
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

#Hyper-parameters 
k = 4 
batch_size = 16
num_epochs = 150


#Create dictionary entries of all data. 
py_train_path = 'gdrive/My Drive/Seizure DL/OverSample/Train/'

partition = {}
partition['train'] = np.array(os.listdir(py_train_path))


labels = {}
for p in partition:
    for item in partition[p]:
        if 'seizure' in item:
            labels[item] = 1
        else:
            labels[item] = 0


params ={'location': py_train_path,
        'dim': (80,50),
          'batch_size': batch_size,
          'n_classes': 1,
          'n_channels': 1,
          'shuffle': True}

validation_params ={'location': py_train_path,
        'dim': (80,50),
          'batch_size': 62,
          'n_classes': 1,
          'n_channels': 1,
          'shuffle': True}
        



#Training at a rediculuously large number of epochs so we can plot overfitting. 
all_acc_histories = []
all_loss_histories = []
all_f1_histories = []

num_val_samples = len(partition['train']) // k 
print(num_val_samples)

random.shuffle(partition['train'])

model_histories = []
save_models = []
save_data = [] #[ (train_data, val_data), ... ]


for i in range(k): 
    #Partition entire dataset into training and testing
    val_data = partition['train'][i*num_val_samples: (i+1) * num_val_samples]
    print('length of validation data:' + str(len(val_data)))
    partial_train_data = np.concatenate([partition['train'][:i*num_val_samples], partition['train'][(i+1)*num_val_samples:]], axis=0)

    save_data.append((partial_train_data,val_data))

    #Create data generators for the train and test set.
    training_generator = DataGenerator(partial_train_data, labels, **params)
    validating_generator = DataGenerator(val_data, labels, **validation_params)
    
    model = build_model_2()
    model.summary()

    root = 'gdrive/My Drive/Seizure DL/OverSample/'

    mc = ModelCheckpoint(filepath=root +'weights.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True)
    callbacks = [mc]

    history = model.fit_generator(generator=training_generator, validation_data=validating_generator, validation_freq = 10, use_multiprocessing=True, workers=4, epochs=num_epochs, callbacks = callbacks, verbose=1)
    model_histories.append(history)
    save_models.append(model)

    acc_history = history.history['acc']
    loss_history = history.history['loss']
    f1_history = history.history['f1_score'] 
    
    all_loss_histories.append(loss_history)
    all_acc_histories.append(acc_history)
    all_f1_histories.append(f1_history)




In [0]:
#Evaluate model.
path = 'gdrive/My Drive/Seizure DL/OverSample/Test/'

(test_x, test_y) = load_data(path)

pred = save_models[3].predict(test_x)
save_models[3]
for i,p in enumerate(pred):
    print(str(p) + ' : ' + str(test_y[i]))

## Fit the model

In [0]:
import numpy as np
import random 
import os 

#For no more tensor flow warnings. 
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')


In [0]:
#Train model with oversampled dataset. 
import random
import os
import numpy as np
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers

#HyperParameters
batch_size = 16
num_epochs = 450


py_path = 'gdrive/My Drive/Seizure DL/OverSample_2/Train/'
py_path_test = 'gdrive/My Drive/Seizure DL/OverSample_2/Test/'

partition = {}
partition['train'] = np.array(os.listdir(py_path))
partition['test'] = np.array(os.listdir(py_path_test))


#Debug. 
print('Length of training set: ' + str(len(partition['train'])))
# print('Length of testing set: ' + str(len(partition['test'])))

labels = {}
for p in partition:
    for item in partition[p]:
        if 'seizure' in item:
            labels[item] = 1
        else:
            labels[item] = 0

_count = 0
for item in partition['train']:
    if 'seizure' in item:
        _count += 1
print('Number of Seizure videos in training set: ' + str(_count) + ', ' + str(_count / len(partition['train'])) + '%')

random.shuffle(partition['train'])

#Split up training for testing and validation. 
_size = len(partition['train']) // 8

train_data = partition['train']
validation_data = partition['test']
test_data = partition['test']

#Diagnostics on training and testing sets. 
s_count = 0
for td in train_data:
    s_count += labels[td]

test_count = 0
for td in test_data:
    test_count+= labels[td]

valid_count = 0
for td in validation_data:
    valid_count += labels[td]

print('Training Data:\n' + 'Total Videos: ' + str(len(train_data)) + '\nSeizure Videos: ' + str(s_count))
print('\nValidation Data:\n' + 'Total Videos: ' + str(len(validation_data)) + '\nSeizure Videos: ' + str(valid_count))
print('\nTesting Data:\n' + 'Total Videos: ' + str(len(test_data)) + '\nSeizure Videos: ' + str(test_count))

params ={'location': py_path,
        'dim': (80,50),
          'batch_size': batch_size,
          'n_classes': 1,
          'n_channels': 1,
          'shuffle': True}

test_params ={'location': py_path,
        'dim': (80,50),
          'batch_size': len(test_data),
          'n_classes': 1,
          'n_channels': 1,
          'shuffle': True}

validation_params ={'location': py_path_test,
        'dim': (80,50),
          'batch_size': len(validation_data),
          'n_classes': 1,
          'n_channels': 1,
          'shuffle': True}

training_generator = DataGenerator(train_data, labels, **params)

# testing_generator = DataGenerator(test_data, labels, **test_params)

validating_generator = DataGenerator(validation_data, labels, **validation_params)

root = 'gdrive/My Drive/Seizure DL/OverSample_2/'

#Build Model
model = build_model_3()
model.summary()
#from keras.callbacks import ModelCheckpoint
# mc = ModelCheckpoint(filepath=root +'weights.{epoch:02d}.hdf5', monitor='val_recall_m', mode='max', save_best_only=True)
mc_f1 = ModelCheckpoint(filepath=root +'weights.{epoch:02d}.hdf5', monitor='val_loss', mode='min', save_best_only=True)
# es = EarlyStopping(monitor='val_loss', baseline=5, verbose=1)
call_backs = [mc_f1]
#Fit model. 
history = model.fit_generator(generator=training_generator, validation_data=validating_generator, callbacks=call_backs, use_multiprocessing=True, workers=8, epochs=num_epochs, verbose=1)


In [0]:
#Print model.
from keras.utils import plot_model
plot_model(model, to_file='simple_model.png')
model.summary()

In [0]:
def smooth_curve(points, factor=.9):
  smoothed_points = []
  for point in points: 
    #print(point)
    if smoothed_points: 
      previous = smoothed_points[-1]
      #print(previous)
      smoothed_points.append(previous*factor + point * (1-factor))
    else:
      smoothed_points.append(point)
    
  return smoothed_points

import matplotlib.pyplot as plt
plt.figure()
val_loss = smooth_curve(history.history['val_loss'])
loss = smooth_curve(history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
num_epochs = range(345)

plt.plot(num_epochs, val_loss[:len(num_epochs)], 'r', label = 'Val Loss')
plt.plot( num_epochs, loss[:len(num_epochs)], 'b', label = 'Train Loss')
plt.legend()
plt.show()

plt.figure()
val_loss = smooth_curve(history.history['val_precision_m'])
loss = smooth_curve(history.history['precision_m'])
plt.xlabel('epochs')
plt.ylabel('Precision')
num_epochs = range(450)


plt.plot(num_epochs, val_loss[:len(num_epochs)], 'r', label = 'Val Precision')
plt.plot( num_epochs, loss[:len(num_epochs)], 'b', label = 'Train Precision')
plt.legend()
plt.show()



# Load Model

In [0]:
import numpy as np
import random 
import os 

#For no more tensor flow warnings. 
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')
from keras import models

#Load past model and evaluate the model
model_path = 'gdrive/My Drive/Seizure DL/OverSample_2/weights.18.hdf5'
train_data_path = 'gdrive/My Drive/Seizure DL/OverSample_2/Train/'
test_data_path = 'gdrive/My Drive/Seizure DL/OverSample_2/Test/'

#Load model 
dependencies = {
    'f1_score': f1_score,
    'precision_m': precision_m,
    'recall_m': recall_m
}
# model = keras.models.load_model(self.output_directory + 'best_model.hdf5', custom_objects=dependencies)
root = 'gdrive/My Drive/Seizure DL/OverSample_2/'
# model_paths = ['weights.161.hdf5','weights.162.hdf5','weights.211.hdf5']
model_paths = ['weights.340.hdf5']

for mp in model_paths:
    print('\n')
    print(mp)
    # my_model = build_model_3()
    my_model = load_model(root+mp, custom_objects=dependencies)
    #Load testing data
    (train_x, train_y, train_names), (test_x, test_y, test_names) = load_data(train_location=train_data_path, test_location=test_data_path, get_name=True)


    h = my_model.evaluate(test_x, test_y, verbose=1)

    for i in range(len(test_y)):
        pred = my_model.predict(x=test_x[i].reshape(1,80,50))
        print(test_names[i] + ' : ' + str(pred))

    print('test output:')
    print(h)
